In [1]:
import pandas as pd

In [ ]:
cuituis = pd.read_csv("CUI-TUIs.csv", engine="pyarrow")
cuicuis = pd.read_csv("CUI-CUIs.csv", engine="pyarrow")
suis = pd.read_csv("SUIs.csv", engine="pyarrow")
cuisuis = pd.read_csv("CUI-SUIs.csv", engine="pyarrow")


In [7]:
# cuituis.rename(columns={":START_ID": "CUI", ":END_ID": "TUI"}, inplace=True)
cuituis.rename(columns={"cui": "CUI", "tui": "TUI"}, inplace=True)
cuituis.head()

,CUI,TUI
0,C0541479,T104
1,C0541480,T104
2,C0541481,T104
3,C0070474,T104
4,C0541516,T104


In [8]:
cuicuis.rename(columns={":START_ID": "CUI1", ":END_ID": "CUI2", ":TYPE": "RELATION"}, inplace=True)
cuicuis.head()

,CUI1,CUI2,RELATION,SAB
0,C0270715,C0236642,RB,AOD
1,C0037728,C0003787,RB,AOD
2,C0032636,C0018090,RB,AOD
3,C0024264,C0039194,RB,AOD
4,C0024264,C0004561,RB,AOD


In [9]:
cuisuis.rename(columns={":START_ID": "CUI", ":END_ID": "SUI"}, inplace=True)
cuisuis.head()

,CUI,SUI
0,C0011251,S0000118
1,C0041672,S0000155
2,C0030540,S0000220
3,C0035861,S0000313
4,C0041189,S0000327


In [11]:
suis.rename(columns={"SUI:ID": "SUI", 'name': 'NAME'}, inplace=True)
suis.head()

,SUI,NAME
0,S0000001,Mild mental retardation
1,S0000002,Moderate mental retardation
2,S0000003,Severe mental retardation
3,S0000004,Profound mental retardation
4,S0000005,Unspecified mental retardation


In [13]:
cui_to_sui = dict(zip(cuisuis['CUI'], cuisuis['SUI']))
sui_to_name = dict(zip(suis['SUI'], suis['NAME']))
cuicuis['SUI1'] = cuicuis['CUI1'].map(cui_to_sui)
cuicuis['SUI2'] = cuicuis['CUI2'].map(cui_to_sui)
cuicuis.head()

,CUI1,CUI2,RELATION,SAB,SUI1,SUI2
0,C0270715,C0236642,RB,AOD,S1677791,S1675807
1,C0037728,C0003787,RB,AOD,S0087060,S0000766
2,C0032636,C0018090,RB,AOD,S0075165,S8927864
3,C0024264,C0039194,RB,AOD,S0002264,S0091265
4,C0024264,C0004561,RB,AOD,S0002264,S0017400


In [14]:
cuicuis['NAME1'] = cuicuis['SUI1'].map(sui_to_name)
cuicuis['NAME2'] = cuicuis['SUI2'].map(sui_to_name)
cuicuis.head()

,CUI1,CUI2,RELATION,SAB,SUI1,SUI2,NAME1,NAME2
0,C0270715,C0236642,RB,AOD,S1677791,S1675807,"Degenerative Diseases, Central Nervous System",Pick Disease of the Brain
1,C0037728,C0003787,RB,AOD,S0087060,S0000766,Southwestern United States,Arizona
2,C0032636,C0018090,RB,AOD,S0075165,S8927864,Pongidae,Gorilla <Western Gorilla>
3,C0024264,C0039194,RB,AOD,S0002264,S0091265,Lymphocyte,T-Lymphocyte
4,C0024264,C0004561,RB,AOD,S0002264,S0017400,Lymphocyte,B-Lymphocytes


In [15]:
cuicuis.drop(columns=['SUI1', 'SUI2'], inplace=True)
cuicuis.head()

,CUI1,CUI2,RELATION,SAB,NAME1,NAME2
0,C0270715,C0236642,RB,AOD,"Degenerative Diseases, Central Nervous System",Pick Disease of the Brain
1,C0037728,C0003787,RB,AOD,Southwestern United States,Arizona
2,C0032636,C0018090,RB,AOD,Pongidae,Gorilla <Western Gorilla>
3,C0024264,C0039194,RB,AOD,Lymphocyte,T-Lymphocyte
4,C0024264,C0004561,RB,AOD,Lymphocyte,B-Lymphocytes


In [17]:
def get_umls_relation_type(abbreviation: str):
    relation_type_mapping = {
        "AQ": "allowed_qualifier",
        "CHD": "has_child",
        "DEL": "deleted_concept",
        "PAR": "has_parent",
        "QB": "can_be_qualified_by",
        "RB": "has_broader",
        "RL": "is_similar_to",
        "RN": "has_narrower",
        "RO": "has_other_relationship",
        "RQ": "related_possibly_synonymous",
        "RU": "related_unspecified",
        "SY": "source_asserted_synonymy",
        "XR": "not_related_no_mapping"
    }

    return relation_type_mapping.get(abbreviation.upper(), abbreviation)


cuicuis['RELATION'] = cuicuis['RELATION'].apply(get_umls_relation_type)
cuicuis.head()


,CUI1,CUI2,RELATION,SAB,NAME1,NAME2
0,C0270715,C0236642,has_broader,AOD,"Degenerative Diseases, Central Nervous System",Pick Disease of the Brain
1,C0037728,C0003787,has_broader,AOD,Southwestern United States,Arizona
2,C0032636,C0018090,has_broader,AOD,Pongidae,Gorilla <Western Gorilla>
3,C0024264,C0039194,has_broader,AOD,Lymphocyte,T-Lymphocyte
4,C0024264,C0004561,has_broader,AOD,Lymphocyte,B-Lymphocytes


In [18]:
cuicuis.to_parquet("triplets_without_definitions.parquet", index=False)

In [21]:
tuis = pd.read_csv("TUIs.csv", engine="pyarrow")
tuis.head()
tuis.rename(columns={"TUI:ID": "TUI", 'name': 'NAME'}, inplace=True)

In [22]:
cui_to_tui = dict(zip(cuituis['CUI'], cuituis['TUI']))
tui_to_name = dict(zip(tuis['TUI'], tuis['NAME']))
cuicuis['TUI1'] = cuicuis['CUI1'].map(cui_to_tui)
cuicuis['TUI2'] = cuicuis['CUI2'].map(cui_to_tui)
cuicuis['TYPE1'] = cuicuis['TUI1'].map(tui_to_name)
cuicuis['TYPE2'] = cuicuis['TUI2'].map(tui_to_name)
cuicuis.head()

,CUI1,CUI2,RELATION,SAB,NAME1,NAME2,TUI1,TUI2,TYPE1,TYPE2
0,C0270715,C0236642,has_broader,AOD,"Degenerative Diseases, Central Nervous System",Pick Disease of the Brain,T047,T047,Disease or Syndrome,Disease or Syndrome
1,C0037728,C0003787,has_broader,AOD,Southwestern United States,Arizona,T083,T083,Geographic Area,Geographic Area
2,C0032636,C0018090,has_broader,AOD,Pongidae,Gorilla <Western Gorilla>,T015,T015,Mammal,Mammal
3,C0024264,C0039194,has_broader,AOD,Lymphocyte,T-Lymphocyte,T025,T025,Cell,Cell
4,C0024264,C0004561,has_broader,AOD,Lymphocyte,B-Lymphocytes,T025,T025,Cell,Cell


In [23]:
cuicuis.drop(columns=['TUI1', 'TUI2'], inplace=True)
cuicuis.head()

,CUI1,CUI2,RELATION,SAB,NAME1,NAME2,TYPE1,TYPE2
0,C0270715,C0236642,has_broader,AOD,"Degenerative Diseases, Central Nervous System",Pick Disease of the Brain,Disease or Syndrome,Disease or Syndrome
1,C0037728,C0003787,has_broader,AOD,Southwestern United States,Arizona,Geographic Area,Geographic Area
2,C0032636,C0018090,has_broader,AOD,Pongidae,Gorilla <Western Gorilla>,Mammal,Mammal
3,C0024264,C0039194,has_broader,AOD,Lymphocyte,T-Lymphocyte,Cell,Cell
4,C0024264,C0004561,has_broader,AOD,Lymphocyte,B-Lymphocytes,Cell,Cell


In [24]:
cuicuis.to_parquet("triplets_without_definitions--2.parquet", index=False)

In [25]:
cuicuis.head()

,CUI1,CUI2,RELATION,SAB,NAME1,NAME2,TYPE1,TYPE2
0,C0270715,C0236642,has_broader,AOD,"Degenerative Diseases, Central Nervous System",Pick Disease of the Brain,Disease or Syndrome,Disease or Syndrome
1,C0037728,C0003787,has_broader,AOD,Southwestern United States,Arizona,Geographic Area,Geographic Area
2,C0032636,C0018090,has_broader,AOD,Pongidae,Gorilla <Western Gorilla>,Mammal,Mammal
3,C0024264,C0039194,has_broader,AOD,Lymphocyte,T-Lymphocyte,Cell,Cell
4,C0024264,C0004561,has_broader,AOD,Lymphocyte,B-Lymphocytes,Cell,Cell


In [27]:
df = pd.read_parquet("triplets_without_definitions.parquet")
df.head()


,CUI1,CUI2,RELATION,SAB,NAME1,NAME2,TYPE1,TYPE2
0,C0270715,C0236642,has_broader,AOD,"Degenerative Diseases, Central Nervous System",Pick Disease of the Brain,Disease or Syndrome,Disease or Syndrome
1,C0037728,C0003787,has_broader,AOD,Southwestern United States,Arizona,Geographic Area,Geographic Area
2,C0032636,C0018090,has_broader,AOD,Pongidae,Gorilla <Western Gorilla>,Mammal,Mammal
3,C0024264,C0039194,has_broader,AOD,Lymphocyte,T-Lymphocyte,Cell,Cell
4,C0024264,C0004561,has_broader,AOD,Lymphocyte,B-Lymphocytes,Cell,Cell


# CREATING DEFINITIONS PARQUET

In [2]:
import pandas as pd

In [3]:
defs = pd.read_csv("../flat_files_umls_full/DEFs.csv", engine="pyarrow")
defrel = pd.read_csv("../flat_files_umls_full/DEFrel.csv", engine="pyarrow")

In [32]:
defs.rename(columns={"AUI": "ATUI"}, inplace=True)
defs.head()

,ATUI,SAB,DEF
0,AT100258389,MSH,Areas set apart as burial grounds.
1,AT100258390,MSH,A non-metal element that has the atomic symbol...
2,AT100258391,MSH,A genus of ectomycorrhizae basidiomycetous fun...
3,AT100258392,MSH,"A republic in southern Africa, south of TANZAN..."
4,AT100258393,MSH,The flow of ions into or out of cells that cau...


In [33]:
defrel.rename(columns={":END_ID": "ATUI", ":START_ID": "CUI"}, inplace=True)
defrel.head()

,ATUI,CUI
0,AT100258389,C0007662
1,AT100258390,C0031705
2,AT100258391,C0319858
3,AT100258392,C0026655
4,AT100258393,C2350764


In [34]:
atui_to_cui = dict(zip(defrel['ATUI'], defrel['CUI']))
defs['CUI'] = defs['ATUI'].map(atui_to_cui)
# defs.to_parquet("definitions.parquet", index=False)
defs.head()



,ATUI,SAB,DEF,CUI
0,AT100258389,MSH,Areas set apart as burial grounds.,C0007662
1,AT100258390,MSH,A non-metal element that has the atomic symbol...,C0031705
2,AT100258391,MSH,A genus of ectomycorrhizae basidiomycetous fun...,C0319858
3,AT100258392,MSH,"A republic in southern Africa, south of TANZAN...",C0026655
4,AT100258393,MSH,The flow of ions into or out of cells that cau...,C2350764


In [35]:
defs.drop(columns=['ATUI'], inplace=True)
defs.head()

,SAB,DEF,CUI
0,MSH,Areas set apart as burial grounds.,C0007662
1,MSH,A non-metal element that has the atomic symbol...,C0031705
2,MSH,A genus of ectomycorrhizae basidiomycetous fun...,C0319858
3,MSH,"A republic in southern Africa, south of TANZAN...",C0026655
4,MSH,The flow of ions into or out of cells that cau...,C2350764


In [36]:
defs.to_parquet("definitions.parquet", index=False)

In [37]:
defs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336094 entries, 0 to 336093
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   SAB     336094 non-null  object
 1   DEF     336080 non-null  object
 2   CUI     336094 non-null  object
dtypes: object(3)
memory usage: 7.7+ MB


# JOIN DEF WITH TRIPLETS

In [38]:
triplets = pd.read_parquet("triplets_without_definitions.parquet", engine="pyarrow")
definitions = pd.read_parquet("definitions.parquet", engine="pyarrow")

In [39]:
triplets.head()

,CUI1,CUI2,RELATION,SAB,NAME1,NAME2,TYPE1,TYPE2
0,C0270715,C0236642,has_broader,AOD,"Degenerative Diseases, Central Nervous System",Pick Disease of the Brain,Disease or Syndrome,Disease or Syndrome
1,C0037728,C0003787,has_broader,AOD,Southwestern United States,Arizona,Geographic Area,Geographic Area
2,C0032636,C0018090,has_broader,AOD,Pongidae,Gorilla <Western Gorilla>,Mammal,Mammal
3,C0024264,C0039194,has_broader,AOD,Lymphocyte,T-Lymphocyte,Cell,Cell
4,C0024264,C0004561,has_broader,AOD,Lymphocyte,B-Lymphocytes,Cell,Cell


In [40]:
definitions.head()

,SAB,DEF,CUI
0,MSH,Areas set apart as burial grounds.,C0007662
1,MSH,A non-metal element that has the atomic symbol...,C0031705
2,MSH,A genus of ectomycorrhizae basidiomycetous fun...,C0319858
3,MSH,"A republic in southern Africa, south of TANZAN...",C0026655
4,MSH,The flow of ions into or out of cells that cau...,C2350764


In [41]:
def get_definition(cui: str, df: pd.DataFrame) -> str:
    return df[df['CUI'] == cui]['DEF'].values[0]

triplets['DEF'] = triplets['CUI'].map(get_definition, args=(definitions,))
triplets.head()
# triplets.to_parquet("triplets_with_definitions.parquet", index=False)


KeyError: 'CUI'